In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
hist = utils.read_df_pkl('../input/historical_transactions0*')
new = utils.read_df_pkl('../input/new*0*')
df = pd.concat([hist, new], axis=0)

100%|██████████| 3/3 [00:00<00:00,  3.40it/s]


In [3]:
dm = pd.read_csv('../input/merchant_id_outlier_purchase_amount.csv')
### cnt_idが3000以上のmechant_id
dm = dm[dm.cnt_id>=3000]
dm.head()

,merchant_id,outlier,purchase_amount_mean,purchase_amount_max,cnt_id
0,M_ID_57df19bf28,0.038039,-0.702909,-0.281537,4548
1,M_ID_6f274b9340,0.027521,-0.722065,0.092984,4542
2,M_ID_445742726b,0.026094,-0.717119,4.072095,5097
3,M_ID_9e84cda3b1,0.024209,-0.458707,3.712961,3635
4,M_ID_86be58d7e0,0.022887,-0.686391,-0.287547,5986


In [4]:
# outlierが0.01以上
out_merchant_list = dm[dm.outlier>0.01].merchant_id.values.tolist()

# purchase_amount_meanが0以上
high_merchant_list = dm[dm.purchase_amount_mean>0].merchant_id.values.tolist()

# depart系
depart_merchant_list = dm[dm.purchase_amount_max>30][dm.purchase_amount_mean<-0.5].merchant_id.values.tolist()

merchant_list = list(set(out_merchant_list + high_merchant_list + depart_merchant_list))
len(merchant_list)

60

In [5]:
df_m = df.loc[df.merchant_id.isin(merchant_list), ['merchant_id', 'card_id']]
print(df_m.shape)
df_m.head()

(4330696, 2)


,merchant_id,card_id
0,M_ID_e020e9b302,C_ID_4e6213e9bc
1,M_ID_86ec983688,C_ID_4e6213e9bc
4,M_ID_e020e9b302,C_ID_4e6213e9bc
6,M_ID_5e8220e564,C_ID_4e6213e9bc
12,M_ID_e020e9b302,C_ID_4e6213e9bc


In [16]:
# tmp = df_m.drop_duplicates().set_index(key)
# ohe = pd.get_dummies(data=tmp).reset_index()
ohe = ohe.groupby(key).max().reset_index()
ohe.head()

,card_id,merchant_id_M_ID_00a6ca8a8a,merchant_id_M_ID_076cd8a829,merchant_id_M_ID_09d7a200d1,merchant_id_M_ID_1ac6bbc867,merchant_id_M_ID_1f4773aa76,merchant_id_M_ID_2637773dd2,merchant_id_M_ID_26d4fadb60,merchant_id_M_ID_29ec8d2ab8,merchant_id_M_ID_2c4b8c61f6,merchant_id_M_ID_3111c6df35,merchant_id_M_ID_445742726b,merchant_id_M_ID_48257bb851,merchant_id_M_ID_4e461f7e14,merchant_id_M_ID_50f575c681,merchant_id_M_ID_57df19bf28,merchant_id_M_ID_5ba019a379,merchant_id_M_ID_5cea7e02a8,merchant_id_M_ID_5e8220e564,merchant_id_M_ID_620909e2a7,merchant_id_M_ID_69f80607d1,merchant_id_M_ID_6f274b9340,merchant_id_M_ID_77197a9b5e,merchant_id_M_ID_77e2942cd8,merchant_id_M_ID_79692349d6,merchant_id_M_ID_82a30d9203,merchant_id_M_ID_86be58d7e0,merchant_id_M_ID_86ec983688,merchant_id_M_ID_879f625c89,merchant_id_M_ID_8c6b897b35,merchant_id_M_ID_8fadd601d2,merchant_id_M_ID_8fcb16a959,merchant_id_M_ID_9139332ccc,merchant_id_M_ID_940fb4498f,merchant_id_M_ID_96ba3c7b60,merchant_id_M_ID_97f0959810,merchant_id_M_ID_9dbbaabb0a,merchant_id_M_ID_9e84cda3b1,merchant_id_M_ID_a08fe74b06,merchant_id_M_ID_a9430074ad,merchant_id_M_ID_b7459adc65,merchant_id_M_ID_b794b9d9e8,merchant_id_M_ID_b9dcf28cb9,merchant_id_M_ID_b9f9332438,merchant_id_M_ID_c03b62d83d,merchant_id_M_ID_c355870ab2,merchant_id_M_ID_c9855bf3e3,merchant_id_M_ID_ca4d18b336,merchant_id_M_ID_cc72dac895,merchant_id_M_ID_cd2c0b07e9,merchant_id_M_ID_d13262f4b0,merchant_id_M_ID_de40fbd87c,merchant_id_M_ID_deb43ff012,merchant_id_M_ID_e020e9b302,merchant_id_M_ID_e5374dabc0,merchant_id_M_ID_ee75252e6a,merchant_id_M_ID_eeba6b925d,merchant_id_M_ID_ef87dd1879,merchant_id_M_ID_f40ccc1312,merchant_id_M_ID_f4be79819d,merchant_id_M_ID_fc7d7969c3
0,C_ID_00007093c1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,C_ID_0001238066,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2,C_ID_0001506ef0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,C_ID_0001793786,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,C_ID_000183fdda,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
base = utils.read_df_pkl('../input/base*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

df_train = base_train.merge(ohe, how='left', on=key)
df_test = base_test.merge(ohe, how='left', on=key)

for col in df_train.columns:
    if col in ignore_list:continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/114_mer_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/114_mer_test_{col}@', obj=df_test[col].values)

100%|██████████| 3/3 [00:00<00:00, 84.55it/s]
